# HW3 Image Classification

## Get Data

### Download Data

In [13]:
# !curl -L "https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip" -o food11.zip


### Manually Unzip

## Preparation

### Importing

In [14]:
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

from tqdm.auto import tqdm, trange
import random

from torch.utils.tensorboard import SummaryWriter


### Transforms

In [15]:
test_tfm = transforms.Compose(
    [transforms.Resize((128, 128)),
     transforms.ToTensor()])

train_tfm = transforms.Compose(
    [transforms.Resize((128, 128)),
     transforms.ToTensor()])


### Define Dataset

In [ ]:
from fileinput import filename


class FoodDataset(Dataset):

    def __init__(self, path, tfm=test_tfm, files=None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([
            os.path.join(path, x)
            for x in os.listdir(path)
            if x.endswith(".jpg")
        ])
        if files:
            self.files = files
        print(f"One {path} sample", self.files[0])
        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        file_name = self.files[index]
        image = Image.open(file_name)
        image = self.transform(image)

        try:
            # name of an image:  .../[label]_[id].jpg
            base_name = os.path.basename(file_name)
            label = int(base_name.split("_")[0])
        except:
            return label
        return image, label


### Define Model

#### Define CNN Block

In [17]:
class CNN_Block(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1):
        super(CNN_Block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size=kernel_size,
                      stride=stride,
                      padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)  # 2x2 maxpool
        )

    def forward(self, x):
        return self.block(x)


#### Define Classifier

In [18]:
class Classifier(nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        # input dim: [3, 128, 128]
        self.cnn = nn.Sequential(
            CNN_Block(3, 64),
            # [64, 64, 64]
            CNN_Block(64, 128),
            # [128, 32, 32]
            CNN_Block(128, 256),
            # [256, 16, 16]
            CNN_Block(256, 512),
            # [512, 8, 8]
            CNN_Block(512, 512)
            # [512, 4, 4]
        )

        self.fc = nn.Sequential(nn.Linear(512 * 4 * 4, 512), nn.ReLU(),
                                nn.Linear(512, 1024), nn.ReLU(),
                                nn.Linear(1024, 1024), nn.ReLU(),
                                nn.Linear(1024, 11))

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


### Hyperparameters

In [19]:
# device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# training parameters
# random seed
seed = 3407

num_epoch = 500
early_stop = 5

batch_size = 64
learning_rate = 1e-4
weight_decay = 1e-5

dataset_dir = './food11'
model_path = './model.ckpt'


cuda


### Fixing seed

In [20]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


same_seeds(seed)


### Model

In [21]:
model = Classifier().to(device)


### Optimizer


In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=weight_decay)


### Dataset

In [23]:
train_set = FoodDataset(os.path.join(dataset_dir, "training"), tfm=train_tfm)
train_loader = DataLoader(train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0,
                          pin_memory=True)
valid_set = FoodDataset(os.path.join(dataset_dir, "validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0,
                          pin_memory=True)


One ./food11\training sample ./food11\training\0_0.jpg
One ./food11\validation sample ./food11\validation\0_0.jpg


## Training

In [24]:
best_acc = 0.0
not_improving = 0

pbar = trange(num_epoch)

for epoch in pbar:

    # training
    train_acc = 0.0
    train_loss = 0.0
    model.train()

    for batch in tqdm(train_loader, leave=False, desc="Training"):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        # Clip the gradient norms for stable training.
        # grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        optimizer.step()

        # torch.max -> max_values, max_value_indices
        train_pred = outputs.argmax(dim=-1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    # vaildation
    valid_acc = 0.0
    valid_loss = 0.0
    model.eval()

    with torch.no_grad():
        for batch in tqdm(valid_loader, leave=False, desc='Validation'):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)

            val_pred = outputs.argmax(dim=-1)
            valid_acc += (val_pred.cpu() == labels.cpu()).sum().item()
            valid_loss += loss.item()

    train_acc /= len(train_set)
    train_loss /= len(train_loader)
    valid_acc /= len(valid_set)
    valid_loss /= len(valid_loader)

    # save best model on validation
    if valid_acc > best_acc:
        not_improving = 0
        best_acc = valid_acc
        torch.save(model.state_dict(), model_path)
    else:
        not_improving += 1

    # display
    info_str = f"Best Acc: {best_acc:.4f}  " + f"Last Training Acc: {train_acc:.4f}  " + f"Last Training Loss: {train_loss:.4f}  " + f"Last Validation Acc: {valid_acc:.4f}  " + (
        f"Not Improved for {not_improving} epoches" if not_improving else "")
    pbar.set_postfix_str(info_str)

    if not_improving >= early_stop:
        break


  0%|          | 0/500 [00:00<?, ?it/s]

Training:   0%|          | 0/155 [00:00<?, ?it/s]

Validation:   0%|          | 0/54 [00:00<?, ?it/s]

Training:   0%|          | 0/155 [00:00<?, ?it/s]

Validation:   0%|          | 0/54 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/54 [00:00<?, ?it/s]

Training:   0%|          | 0/155 [00:00<?, ?it/s]

Validation:   0%|          | 0/54 [00:00<?, ?it/s]

Training:   0%|          | 0/155 [00:00<?, ?it/s]

Validation:   0%|          | 0/54 [00:00<?, ?it/s]

# Test & Predict

In [25]:
test_set = FoodDataset(os.path.join(dataset_dir, "test"), tfm=test_tfm)
test_loader = DataLoader(test_set,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=0,
                         pin_memory=True)


One ./food11\test sample ./food11\test\0001.jpg


In [26]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load("model.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data, _ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()


Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label found!
Invalid label

In [27]:
#create test csv
def pad4(i):
    return "0" * (4 - len(str(i))) + str(i)


df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1, len(test_set) + 1)]
df["Category"] = prediction
df.to_csv("submission.csv", index=False)
